# 1 求未知数字

In [ ]:
w = [7, 3, 1] * 10
letter2num = {chr(ord('A') + i): i+10 for i in range(26)}
letter2num.update({str(i): i for i in range(9)})
letter2num['<'] = 0
letter2num

In [ ]:
# 计算校验和
# 12345678<8<<<1110182<111116?<<<<<<<<<<<<<<<4
s = "12345678<8<<<1110182<111116"

for guess in range(9):
    teps = s + str(guess)
    checksum = 0
    for idx, ch in enumerate(teps):
        checksum += letter2num[ch] * w[idx]
    checksum = checksum % 10
    if checksum == 4:
        print(guess)

# 结果是 7

# 2 求key

In [ ]:
import codecs
from hashlib import sha1
import base64
import binascii
from Crypto.Cipher import AES

def cal_Kseed() -> str:
    MRZ_information = "12345678<811101821111167"  # 护照信息
    H_information = sha1(MRZ_information.encode()).hexdigest()  # 使用SHA1进行哈希
    K_seed = H_information[0:32]  # 取哈希值的前32位作为K_seed
    return K_seed

def cal_Ka_Kb(K_seed):
    c = "00000001"
    d = K_seed + c
    H_d = sha1(codecs.decode(d, "hex")).hexdigest()  # 对K_seed进行哈希
    ka = H_d[0:16]  # 取前16位作为ka
    kb = H_d[16:32]  # 取后16位作为kb
    return ka, kb

def Parity_Check(x):
    k_list = []
    a = bin(int(x, 16))[2:]  # 将16进制转为2进制
    for i in range(0, len(a), 8):
        # 7位一组分块，计算一个校验位，使1的个数为偶数
        if (a[i:i + 7].count("1")) % 2 == 0:
            k_list.append(a[i:i + 7])
            k_list.append('1')
        else:
            k_list.append(a[i:i + 7])
            k_list.append('0')
    k = hex(int(''.join(k_list), 2))  # 将2进制转为16进制
    return k

In [ ]:
ka, kb = cal_Ka_Kb(cal_Kseed())
k1, k2 = Parity_Check(ka), Parity_Check(kb)
key = k1[2:] + k2[2:]  # 合并k_1和k_2作为最终的密钥

In [ ]:
ciphertext = base64.b64decode(
    "9MgYwmuPrjiecPMx61O6zIuy3MtIXQQ0E59T3xB6u0Gyf1gYs2i3K9Jxaa0zj4gTMazJuApwd6+jdyeI5iGHvhQyDHGVlAuYTgJrbFDrfB22Fpil2NfNnWFBTXyf7SDI")
IV = '0' * 32  # 初始化向量

# 使用AES进行解密
m = AES.new(binascii.unhexlify(key), AES.MODE_CBC, binascii.unhexlify(IV)).decrypt(ciphertext)
print(m)  # 输出解密后的明文